In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
import nltk
nltk.download('stopwords')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
url ="/content/drive/MyDrive/Colab Notebooks/data/sms/spam.csv"
data = pd.read_csv(url,encoding='latin-1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
#EDA

data.describe()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
count,5572,5572,50,12,6
unique,2,5169,43,10,5
top,ham,"Sorry, I'll call later","bt not his girlfrnd... G o o d n i g h t . . .@""","MK17 92H. 450Ppw 16""","GNT:-)"""
freq,4825,30,3,2,2


In [ ]:
data.columns

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [ ]:
data.isnull().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [ ]:
data.dropna(how="any",axis=1,inplace = True)

In [ ]:
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
# Renaming column names
data.columns = ["Category","Message"]
# Encoding target variable
data.replace(['ham','spam'],[0, 1])

data = data.reset_index(drop=True)

In [ ]:
# Count the number of words in each Text in the dataset
data['Count']=0
for i in np.arange(0,len(data.Message)):
    data.loc[i,'Count'] = len(data.loc[i,'Message'])

data.head()

,Category,Message,Count
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [ ]:
data['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [ ]:
corpus = []
ps = PorterStemmer()

In [ ]:
#Little bit of refferance taken from internet

for i in range(0, 5572):

    # Applying Regular Expression
    
    '''
    Replace email addresses with 'emailadd'
    Replace URLs with 'httpadd'
    Replace money symbols with 'Currencysymbol'
    Replace phone numbers with 'phonenumber'
    Replace numbers with 'number'
    '''
    msg = data['Message'][i]
    msg = re.sub('\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailadd', data['Message'][i])
    msg = re.sub('(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpadd', data['Message'][i])
    msg = re.sub('£|\$', 'Currencysymbol', data['Message'][i])
    msg = re.sub('\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', 'phonenumber', data['Message'][i])
    msg = re.sub('\d+(\.\d+)?', 'number', data['Message'][i])
    
    ''' Remove all punctuations '''
    msg = re.sub('[^\w\d\s]', ' ', data['Message'][i])
    
    if i<2:
        print("\t\t\t\t MESSAGE ", i)
    
    if i<2:
        print("\n After Regular Expression - Message ", i, " : ", msg)
    
    # Each word to lower case
    msg = msg.lower()    
    if i<2:
        print("\n Lower case Message ", i, " : ", msg)
    
    # Splitting words to Tokenize
    msg = msg.split()    
    if i<2:
        print("\n After Splitting - Message ", i, " : ", msg)
    
    # Stemming with PorterStemmer handling Stop Words
    msg = [ps.stem(word) for word in msg if not word in set(stopwords.words('english'))]
    if i<2:
        print("\n After Stemming - Message ", i, " : ", msg)
    
    # preparing Messages with Remaining Tokens
    msg = ' '.join(msg)
    if i<2:
        print("\n Final Prepared - Message ", i, " : ", msg, "\n\n")

    # Preparing WordVector Corpus
    corpus.append(msg)

				 MESSAGE  0

 After Regular Expression - Message  0  :  Go until jurong point  crazy   Available only in bugis n great world la e buffet    Cine there got amore wat   

 Lower case Message  0  :  go until jurong point  crazy   available only in bugis n great world la e buffet    cine there got amore wat   

 After Splitting - Message  0  :  ['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']

 After Stemming - Message  0  :  ['go', 'jurong', 'point', 'crazi', 'avail', 'bugi', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'got', 'amor', 'wat']

 Final Prepared - Message  0  :  go jurong point crazi avail bugi n great world la e buffet cine got amor wat 


				 MESSAGE  1

 After Regular Expression - Message  1  :  Ok lar    Joking wif u oni   

 Lower case Message  1  :  ok lar    joking wif u oni   

 After Splitting - Message  1  :  ['ok', 'lar', 'joking', 'wif', '

In [ ]:
#Intitializing vectorizer
cv = CountVectorizer()
# Intitializing target and features
X = cv.fit_transform(corpus).toarray()
y = data['Category']
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
# Initializing train test split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size= 0.20, random_state = 7)

In [ ]:
#Building model

#Initializing naive bayes classfier
bayes_classifier = GaussianNB()
bayes_classifier.fit(X_train, y_train)


GaussianNB()

In [ ]:
#Initializing Decision Tree Classfier
Dtc = DecisionTreeClassifier(random_state=7)
Dtc.fit(X_train, y_train)

DecisionTreeClassifier(random_state=7)

In [ ]:
y_pred_nb = bayes_classifier.predict(X_test)

In [ ]:
y_pred_dt = Dtc.predict(X_test)

In [ ]:
#Using confusion matrix to evaluate the model
confusionnb = confusion_matrix(y_test, y_pred_nb)

In [ ]:
confusionDtc = confusion_matrix(y_test, y_pred_dt)

In [ ]:
#Accuracy and classsification report for nb
#print ("Accuracy :", accuracy_score(y_test, y_pred_nb))
print (classification_report(y_test, y_pred_nb))

              precision    recall  f1-score   support

           0       0.98      0.88      0.93       970
           1       0.53      0.90      0.66       145

    accuracy                           0.88      1115
   macro avg       0.76      0.89      0.80      1115
weighted avg       0.92      0.88      0.89      1115



In [ ]:
#Accuracy and classsification report for decision Tree
print ("Accuracy :", accuracy_score(y_test, y_pred_dt))
print (classification_report(y_test, y_pred_dt))

Accuracy : 0.9739910313901345
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       970
           1       0.96      0.83      0.89       145

    accuracy                           0.97      1115
   macro avg       0.97      0.91      0.94      1115
weighted avg       0.97      0.97      0.97      1115

